# Covid19 Workflow

This is the interactive version of the Covid19 workflow using PyCOMPSs.

It uses the Building Blocks from jupyter, whilst PyCOMPSs is used to parallelize the workflow.

In [ ]:
import os
import csv
import glob
from pathlib import Path

current_path = os.getcwd()
if not "PERMEDCOE_IMAGES" in os.environ:
    os.environ["PERMEDCOE_IMAGES"] = current_path + "/../../../../BuildingBlocks/Resources/images/"
os.environ["COMPUTING_UNITS"] = "1"

First, start the PyCOMPSs runtime and import the used PyCOMPSs synchronization function (``compss_wait_on_directory``)

In [ ]:
import pycompss.interactive as ipycompss

In [ ]:
ipycompss.start(graph=True, trace=False, debug=True)

# PyCOMPSs imports
from pycompss.api.api import compss_wait_on_directory
from pycompss.api.api import compss_wait_on_file

Second, import the required Building Blocks

In [ ]:
# To set the default PyCOMPSs TMPDIR
from permedcoe import TMPDIR

# Import building block tasks
from MaBoSS_BB import MaBoSS_analysis
from single_cell_processing_BB import single_cell_processing
from personalize_patient_BB import personalize_patient
from PhysiBoSS_BB import physiboss_model, physiboss_analyse_replicates
from meta_analysis_BB import meta_analysis

Third, set the input parameters and load input data

In [ ]:
print("--------------------")
print("| Covid19 Workflow |")
print("--------------------")

arg_metadata = current_path + "/../../../Resources/data/metadata_small_local.tsv"
arg_model_prefix = current_path + "/../../../Resources/data/epithelial_cell_2"
arg_outdir = current_path + "/results/"
arg_ko_file = current_path + "/ko_file.txt"
arg_reps = 2
arg_model = "epithelial_cell_2"
arg_data_folder = current_path + "/../../../Resources/data/"
arg_max_time = 100

In [ ]:
if os.path.exists(arg_outdir):
    print("WARNING: the output folder already exists")
else:
    os.makedirs(arg_outdir)

Gene candidates

In [ ]:
if os.path.exists(arg_ko_file):
    print("KO file provided")
else:
    print("KO file not detected, running MABOSS")
    ## MABOSS
    # This step produces the ko_file.txt, containing the set of selected gene candidates
    MaBoSS_analysis(model=arg_model,
                    data_folder=arg_data_folder,
                    ko_file=arg_ko_file,
                    parallel="1",
                    tmpdir=TMPDIR)
    compss_wait_on_file(arg_ko_file)

Discover gene candidates

In [ ]:
def get_genefiles(prefix, genes):
    """
    Create a list of genes files for the given patient.

    :param prefix: prefix
    :param genes: KO genes
    :return: List of names to be processed
    """
    genefiles = []
    for gene in genes:
        if gene != "":
            name = prefix + "_personalized__" + gene + "_ko"
        else:
            name = prefix + "_personalized"
        genefiles.append(name)
    return genefiles

In [ ]:
genes = [""]  # first empty since it is the original without gene ko
with open(arg_ko_file, "r") as ko_fd:
    genes += ko_fd.read().splitlines()
genefiles = get_genefiles(arg_model, genes)

Iterate over the metadata file processing each patient

In [ ]:
physiboss_results = []
physiboss_subfolder = "physiboss_results"  # do not modify (hardcoded in meta-analysis)
with open(arg_metadata, "r") as metadata_fd:
    reader = csv.DictReader(metadata_fd, delimiter="\t")
    for line in reader:
        # ONE LINE PER PATIENT
        sample = line["id"]
        # SINGLE CELL PROCESSING
        print("> SINGLE CELL PROCESSING %s" % sample)
        sample_out_dir = os.path.join(arg_outdir, sample)
        scp_dir = os.path.join(sample_out_dir, "single_cell_processing", "results")
        os.makedirs(scp_dir)
        scp_images_dir = os.path.join(sample_out_dir, "single_cell_processing", "images")
        os.makedirs(scp_images_dir)
        norm_data = os.path.join(scp_dir, "norm_data.tsv")
        raw_data = os.path.join(scp_dir, "raw_data.tsv")
        scaled_data = os.path.join(scp_dir, "scaled_data.tsv")
        cells_metadata = os.path.join(scp_dir, "cells_metadata.tsv")
        if line["file"].startswith("../.."):
            # Two folder relative - Local
            relative_p_file = os.path.join(*(line["file"].split(os.path.sep)[2:]))  # remove first two folders "../.."
            p_file = os.path.join("..", "..", "Resources", relative_p_file)
        else:
            # Absolute path - Supercomputer
            p_file = line["file"]
        single_cell_processing(tmpdir=TMPDIR,
                               p_id=sample,
                               p_group=line["group"],
                               p_file=p_file,
                               norm_data=norm_data,
                               raw_data=raw_data,
                               scaled_data=scaled_data,
                               cells_metadata=cells_metadata,
                               outdir=scp_images_dir)

        # PERSONALIZE PATIENT
        print("> PERSONALIZING PATIENT %s" % sample)
        pp_dir = os.path.join(sample_out_dir, "personalize_patient")
        os.makedirs(pp_dir)
        model_output_dir = os.path.join(pp_dir, "models")
        personalized_result = os.path.join(pp_dir, "personalized_by_cell_type.tsv")
        personalize_patient(tmpdir=TMPDIR,
                            norm_data=norm_data,
                            cells=cells_metadata,
                            model_prefix=arg_model_prefix,
                            t="Epithelial_cells",
                            model_output_dir=model_output_dir,
                            personalized_result=personalized_result,
                            ko=arg_ko_file)

        for gene_prefix in genefiles:
            print(">> prefix: " + str(gene_prefix))
            for r in range(1, arg_reps + 1):
                print(">>> Repetition: " + str(r))
                name = "output_" + sample + "_" + gene_prefix + "_" + str(r)
                out_file = os.path.join(arg_outdir, sample, physiboss_subfolder, name + ".out")
                err_file = os.path.join(arg_outdir, sample, physiboss_subfolder, name + ".err")
                print("\t- " + out_file)
                print("\t- " + err_file)
                results_dir = os.path.join(arg_outdir, sample, physiboss_subfolder, gene_prefix + "_physiboss_run_" + str(r))
                os.makedirs(results_dir)
                physiboss_results.append(results_dir)
                # PHYSIBOSS
                physiboss_model(sample=sample,
                                repetition=r,
                                prefix=gene_prefix,
                                model_dir=model_output_dir,
                                out_file=out_file,
                                err_file=err_file,
                                results_dir=results_dir,
                                max_time=arg_max_time,
                                tmpdir=TMPDIR)

In [ ]:
# VERSION 1: PROCESS ALL WITHIN THE SAME TASK

# Wait for all physiboss
# Currently needed because the meta analysis requires all of them
# and its input is the main folder. It assumes the internal folder
# structure
for physiboss_result in physiboss_results:
    compss_wait_on_directory(physiboss_result)

physiboss_replicates_results = []
physiboss_replicates_analysis_subfolder = "physiboss_replicates_analysis"  # do not modify (hardcoded in meta-analysis)

with open(arg_metadata, "r") as metadata_fd:
    reader = csv.DictReader(metadata_fd, delimiter="\t")
    for line in reader:
        # ONE LINE PER PATIENT
        sample = line["id"]

        for gene_prefix in genefiles:
            print(">> prefix: " + str(gene_prefix))
            out_file = os.path.join(arg_outdir, sample, physiboss_replicates_analysis_subfolder, gene_prefix + ".out")
            err_file = os.path.join(arg_outdir, sample, physiboss_replicates_analysis_subfolder, gene_prefix + ".err")
            print("\t- " + out_file)
            print("\t- " + err_file)

            physiboss_replicates_result = os.path.join(arg_outdir, sample, physiboss_replicates_analysis_subfolder, gene_prefix + "_results")
            replicates_folder = os.path.join(arg_outdir, sample, physiboss_subfolder)
            prefix = gene_prefix + "_physiboss_run_"

            physiboss_replicates_results.append(physiboss_replicates_result)

            physiboss_analyse_replicates(replicates=arg_reps,
                                        replicates_folder=replicates_folder,
                                        prefix=prefix, 
                                        out_file=out_file,
                                        err_file=err_file,
                                        results_dir=physiboss_replicates_result,
                                        parallel=1,
                                        tmpdir=TMPDIR
            )

# Wait for physiboss
for physiboss_replicates_result in physiboss_replicates_results:
    compss_wait_on_directory(physiboss_replicates_result)

# Perform last step: meta analysis
final_result_dir = os.path.join(arg_outdir, "meta_analysis")
os.makedirs(final_result_dir)
# META-ANALYSIS
meta_analysis(tmpdir=TMPDIR,
              meta_file=arg_metadata,
              out_dir=arg_outdir,
              model_prefix=arg_model,
              ko_file=arg_ko_file,
              reps=arg_reps,
              verbose="T",
              results=final_result_dir)

In [ ]:
compss_wait_on_directory(final_result_dir)

Once the plots have been generate, they can be displayed.

In [ ]:
print("Generated results:")
for res in os.listdir(final_result_dir):
    print(f"\t- {res}")

In [ ]:
from IPython.display import Image
from IPython.display import display

print("Clustermap:")
img = Image(filename=os.path.join(final_result_dir, "clustermap.png"), width = 500, height = 300)
display(img)
print("Clustermap traces:")
img = Image(filename=os.path.join(final_result_dir, "clustermap_traces.png"), width = 500, height = 300)
display(img)
print("Clustermap genes:")
img = Image(filename=os.path.join(final_result_dir, "clustermap_genes.png"), width = 500, height = 300)
display(img)
print("Clustermap patients:")
img = Image(filename=os.path.join(final_result_dir, "clustermap_patients.png"), width = 500, height = 300)
display(img)
print("Dendogram:")
img = Image(filename=os.path.join(final_result_dir, "dendogram.png"), width = 500, height = 300)
display(img)
print("Dendogram traces:")
img = Image(filename=os.path.join(final_result_dir, "dendogram_traces.png"), width = 500, height = 300)
display(img)
print("Dendogram genes:")
img = Image(filename=os.path.join(final_result_dir, "dendogram_genes.png"), width = 500, height = 300)
display(img)
print("Dendogram patients:")
img = Image(filename=os.path.join(final_result_dir, "dendogram_patients.png"), width = 500, height = 300)
display(img)

Finally, we can stop the PyCOMPSs runtime.

In [ ]:
ipycompss.stop(sync=False)